In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re
import html
import string
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import SnowballStemmer

In [2]:
df = pd.read_csv("../data/Youtube-Spam-Dataset.csv")
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,VIDEO_NAME,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",PSY - GANGNAM STYLE(?????) M/V,1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,PSY - GANGNAM STYLE(?????) M/V,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,PSY - GANGNAM STYLE(?????) M/V,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,PSY - GANGNAM STYLE(?????) M/V,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,PSY - GANGNAM STYLE(?????) M/V,1


In [3]:
df_copy = df[['CONTENT', 'CLASS']]
df_copy.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [5]:
import re
from collections import Counter

# Fungsi untuk membersihkan teks komentar
def clean_text(text):
    text = str(text).lower()  # lowercase
    text = re.sub(r"http\S+|www\S+|watch\?v=\S+", "url", text)  # hapus URL
    text = re.sub(r"\[.*?\]", "", text)  # hapus konten dalam tanda kurung []
    text = re.sub(r"<.*?>", "", text)  # hapus tag HTML
    text = re.sub(r"[^\w\s]", "", text)  # hapus tanda baca
    text = re.sub(r"\d+", "num", text)  # hapus angka
    return text.strip()

# Terapkan pembersihan ke kolom CONTENT
df["cleaned"] = df["CONTENT"].apply(clean_text)

# Gabungkan semua kata dari komentar yang sudah dibersihkan
all_words = " ".join(df["cleaned"]).split()

# Hitung total kata dan frekuensi kemunculan
total_words = len(all_words)
word_freq = Counter(all_words)
most_common_words = word_freq.most_common()

total_words, most_common_words


(29372,
 [('i', 748),
  ('this', 729),
  ('the', 713),
  ('and', 632),
  ('to', 632),
  ('out', 584),
  ('my', 581),
  ('you', 574),
  ('check', 559),
  ('a', 550),
  ('num', 496),
  ('on', 387),
  ('it', 338),
  ('is', 330),
  ('of', 309),
  ('video', 305),
  ('song', 280),
  ('for', 263),
  ('me', 246),
  ('please', 242),
  ('like', 241),
  ('subscribe', 225),
  ('in', 222),
  ('youtube', 217),
  ('url', 214),
  ('if', 201),
  ('so', 197),
  ('love', 196),
  ('channel', 193),
  ('just', 168),
  ('music', 149),
  ('can', 147),
  ('that', 137),
  ('have', 135),
  ('up', 125),
  ('but', 123),
  ('your', 123),
  ('new', 122),
  ('guys', 116),
  ('be', 115),
  ('will', 112),
  ('views', 105),
  ('are', 104),
  ('at', 102),
  ('am', 101),
  ('im', 100),
  ('get', 97),
  ('all', 96),
  ('from', 94),
  ('with', 89),
  ('money', 87),
  ('make', 87),
  ('hey', 86),
  ('people', 86),
  ('do', 83),
  ('best', 80),
  ('not', 77),
  ('katy', 77),
  ('who', 75),
  ('comment', 74),
  ('as', 74),
  (

In [6]:
nltk.download('stopwords')

def remove_stopwords(text): 
    """Menghapus stopword dari list kata yang diberikan."""
    stops = set(stopwords.words("english"))
    
    if isinstance(text, list):
        text = [w for w in text if w.lower() not in stops]
        
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Hitung ulang frekuensi kata setelah menghapus stopwords
filtered_word_freq = Counter(remove_stopwords(all_words))
filtered_most_common = filtered_word_freq.most_common()
filtered_total_words = len(remove_stopwords(all_words))

filtered_total_words, filtered_most_common

# Simpan ke CSV
#word_freq_df = pd.DataFrame(filtered_most_common, columns=["word", "frequency"])
#word_freq_df.to_csv(r"..\data\frekuensi_kata_bersih.csv", index=False)

(16936,
 [('check', 559),
  ('num', 496),
  ('video', 305),
  ('song', 280),
  ('please', 242),
  ('like', 241),
  ('subscribe', 225),
  ('youtube', 217),
  ('url', 214),
  ('love', 196),
  ('channel', 193),
  ('music', 149),
  ('new', 122),
  ('guys', 116),
  ('views', 105),
  ('im', 100),
  ('get', 97),
  ('money', 87),
  ('make', 87),
  ('hey', 86),
  ('people', 86),
  ('best', 80),
  ('katy', 77),
  ('comment', 74),
  ('good', 72),
  ('know', 71),
  ('videos', 70),
  ('would', 67),
  ('see', 67),
  ('inumm', 65),
  ('dont', 63),
  ('thank', 63),
  ('u', 62),
  ('much', 62),
  ('thanks', 62),
  ('one', 61),
  ('really', 60),
  ('time', 59),
  ('go', 58),
  ('old', 57),
  ('world', 56),
  ('us', 55),
  ('could', 53),
  ('eminem', 53),
  ('billion', 52),
  ('help', 52),
  ('perry', 52),
  ('everyone', 51),
  ('want', 49),
  ('visit', 49),
  ('called', 49),
  ('songs', 48),
  ('give', 45),
  ('shakira', 45),
  ('free', 43),
  ('share', 41),
  ('still', 41),
  ('today', 41),
  ('mixtape